In [3]:
# take too long to load embedding and match it with pid in each author group, 
# preprocess the embedding speed up the process
import os
# load labeled data pid
Dataset = "pubmed"
fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)
pid_need_extract = []

for file in listfiles:
    if not file.startswith('.'):
        with open(fileDir+file, 'r', encoding = 'utf8') as f:
            for line in f:
                read_data = line.split("\t")
                # some record's doi contain \r or \n character in which creating issue, since we do not use those, ignore it
                if(len(read_data)==13 or len(read_data)==12):
                    pid_need_extract.append(int(read_data[0]))
                else:
                    print(len(read_data))
        f.close()
print("Total labeled records:",len(pid_need_extract))

Total labeled records: 140266


In [4]:
# extract 135796 embedding out of all embedding
pid_need_extract = list(sorted(set(pid_need_extract)))
print("Unique pid: ", len(pid_need_extract))

Unique pid:  135796


In [5]:
# read trained rec to rec tf-idf textual graph
def read_embedding(Dataset = "pubmed", emb_type = "off"):
    emb = []
    while True:
        if emb_type == "pv_dm":
            loadDir = "../Data/"+Dataset+"/vectors/d2v/Doc2Vec(dmm,d100,n5,w5,mc3,s0.001,t24).txt"
            with open(loadDir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    emb.append(read_data)
            f.close()
            print("Total vector records:",len(emb))
            print(emb[0])
            break
        elif emb_type == "pv_dbow":
            loadDir = "../Data/"+Dataset+"/vectors/d2v/Doc2Vec(dbow,d100,n5,mc3,s0.001,t24).txt"
            with open(loadDir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    emb.append(read_data)
            f.close()
            print("Total vector records:",len(emb))
            print(emb[0])
            break
        elif emb_type == "n2v":
            citation_emb_dir = "../Data/"+Dataset+"/vectors/"+emb_type+"/n2v.txt"
            with open(citation_emb_dir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    if(len(read_data)==101):
                        emb.append(read_data)
            f.close()
            print("Total vector records:",len(emb))
            print(emb[:3])
            break
        elif emb_type == "p2v":
            paper_emb_dir = "../Data/"+Dataset+"/vectors/"+emb_type+"/p2v.txt"
            with open(paper_emb_dir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    emb.append(read_data)
            f.close()
            print("Total vector records:",len(emb))
            print(emb[:3])
            break
        elif emb_type == "off":
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="pv_dbow"
    return emb

In [6]:
import pickle
import numpy as np
# read pretrained embeddings
# "pv_dm", "pv_dbow", "n2v"
setting = "p2v"
embedding_type = "p2v"
embeddings = read_embedding(Dataset, embedding_type)

Total vector records: 3149758
[['22516865', '0.0422827', '-0.0443959', '-0.00631522', '-0.0297457', '-0.203585', '0.0791923', '-0.156993', '0.102914', '-0.112826', '0.0301516', '-0.118142', '-0.0447324', '0.0131143', '0.127706', '0.107417', '-0.0843709', '-0.0808756', '-0.0504176', '0.132817', '0.0157529', '-0.0383218', '0.165365', '-0.0619297', '-0.0345336', '0.110633', '0.0860297', '-0.0835764', '0.193302', '0.015641', '0.0323645', '0.00893256', '0.151895', '0.0534986', '-0.18639', '-0.0558457', '-0.0603591', '-0.00530632', '-0.000749852', '0.0772172', '0.0646557', '-0.106445', '-0.028214', '0.0400324', '0.11686', '-0.153841', '-0.064435', '-0.0270715', '-0.0234374', '0.0588498', '-0.0750828', '-0.107259', '-0.0351021', '-0.0166648', '-0.135551', '0.1277', '-0.118618', '-0.0417162', '0.0543777', '0.129501', '-0.152003', '0.0624739', '0.175164', '0.0680417', '-0.0896099', '0.171857', '-0.0123408', '-0.0424403', '-0.017759', '0.0631562', '-0.0947245', '-0.0297129', '-0.0952609', '0.022

In [7]:
print(len(pid_need_extract))
print(len(embeddings))

135796
3149758


In [8]:
embeddings = sorted(embeddings,key=lambda x: (int(x[0])))

In [9]:
print(len(embeddings))
print(embeddings[:3])

3149758
[['3', '-0.0506355', '0.0297007', '-0.174485', '-0.105224', '-0.133456', '0.00829369', '-0.0789215', '-0.109727', '-0.118799', '0.0822599', '-0.0775809', '-0.161325', '-0.165538', '0.13988', '-0.0802623', '0.0257179', '0.131611', '-0.0467805', '0.0317345', '0.0543361', '0.0983858', '-0.0223105', '-0.0280455', '0.149177', '-0.0489583', '0.044019', '0.0107934', '0.0632993', '-0.134811', '0.00876644', '-0.0915887', '0.0680134', '0.0403391', '-0.0370483', '-0.1575', '0.0365223', '0.0593832', '-0.117006', '0.0475719', '-0.0758056', '-0.058571', '-0.0277272', '-0.0375198', '-0.150325', '-0.125805', '-0.0482568', '-0.0809881', '-0.128145', '0.043278', '0.0693474', '0.0494027', '-0.0120701', '0.0232411', '0.0444235', '0.0280239', '-0.180816', '0.0552204', '0.056488', '-0.0354632', '-0.218284', '-0.0715716', '0.00439595', '-0.00320251', '-0.167343', '-0.0307163', '0.0109991', '-0.103033', '-0.281903', '0.136941', '-0.0291817', '0.0569331', '-0.258258', '0.10378', '-0.112591', '0.0180306

In [10]:
# extract needed embedding
wanted_embedding = []
num_fail_extract = 0
for emb in embeddings:
    if(len(pid_need_extract)==0):
        break
    while (pid_need_extract[0]<=int(emb[0])):
        if pid_need_extract[0]==int(emb[0]):
            wanted_embedding.append(emb)
            pid_need_extract.remove(int(emb[0]))
        elif (pid_need_extract[0]<int(emb[0])):
            # remove paper that not in all dataset
            print(pid_need_extract[0], " : ",emb[0])
            pid_need_extract.remove(pid_need_extract[0])
            num_fail_extract+=1
        if len(pid_need_extract)==0:
            break
print("Total paper fail to find: ",num_fail_extract)

24442942  :  24442949
24667040  :  24667042
25060780  :  25060781
25358536  :  25358543
25765001  :  25765011
25800278  :  25800283
26318750  :  26318755
26614021  :  26614022
26616470  :  26616472
26687078  :  26687081
26920701  :  26920703
26922143  :  26922173
26989410  :  26989414
27018814  :  27018820
27077628  :  27077633
27086689  :  27086699
27090087  :  27090089
27135754  :  27135759
27151456  :  27151460
27288420  :  27288435
27399651  :  27399664
27428937  :  27428948
27460683  :  27460692
27500503  :  27500510
27500523  :  27500524
27516149  :  27516160
27524399  :  27524413
27600823  :  27600824
Total paper fail to find:  28


In [11]:
print(len(wanted_embedding))

135768


In [12]:
print(wanted_embedding[:3])

[['8077', '-0.30562', '-0.14224', '-0.295474', '-0.162722', '-0.105769', '0.0561656', '-0.0797713', '-0.0459434', '-0.108158', '0.0544028', '-0.0999472', '-0.0974462', '-0.0508525', '0.151689', '-0.232159', '-0.0359973', '0.0194938', '0.0693435', '-0.112974', '-0.0311077', '0.078018', '-0.0631533', '0.0524421', '0.137697', '0.101945', '-0.0200023', '-0.10946', '0.0944765', '0.00272627', '-0.104189', '-0.0349762', '-0.011765', '0.00508523', '-0.163048', '-0.0154401', '-0.0480973', '0.031805', '-0.0608286', '-0.0115254', '0.0359856', '-0.0689324', '-0.0651454', '0.079306', '-0.0847919', '-0.164983', '-0.172415', '0.0292607', '0.0193611', '-0.0923435', '-0.056113', '0.0077811', '0.0317496', '0.0495394', '-0.139721', '-0.0199847', '-0.0592476', '0.0124314', '0.0360882', '0.134681', '-0.236409', '-0.0483922', '0.0447124', '-0.0540506', '-0.195456', '0.0219141', '0.0426379', '-0.024773', '-0.122287', '0.0582443', '-0.034571', '-0.0649725', '-0.0880582', '0.0407224', '-0.0723574', '0.0289096'

In [13]:
# export needed embedding to txt file
newfileDir = "../Data/"+Dataset+"/vectors/"+setting+"/"
if not os.path.exists(newfileDir):
    os.makedirs(newfileDir)
    
with open(newfileDir+"extracted_labeled_"+embedding_type+".txt", "w",encoding='utf8') as f:
    for emb in wanted_embedding:
        f.write((" ".join(emb)).strip('\n')+"\n")
f.close()